In [3]:
import cv2
import numpy as np   
from keras.preprocessing.image import img_to_array
from keras.models import load_model

from PIL import ImageGrab


# 얼굴 인식 , 감정 분류 트레이닝모델 불러오기
face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('files/emotion_model.hdf5', compile=False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surprising","Neutral"]

# 웹캠을 이용한 카메라 실행 
camera = cv2.VideoCapture(0)
img_counter = 0

#대결할 감정 키워드 입력
print ("화남 = AN , 행복함 = HA, 슬픔 = SA, 놀람 = SU")
emo = input("대결할 감정:")

while True:
    # 카메라 실행
    ret, frame = camera.read()
    if not ret:
        break
    k = cv2.waitKey(1)
    
   
        
    # frame gray scale로 바꿈
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # frame 내 얼굴 감지
    faces = face_detection.detectMultiScale(gray,
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(30,30))
    
    # Create empty image
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    
    # 얼굴이 감지될 때 감정 인식하기
    if len(faces) > 0:
        # 가장 큰 사진 기준으로 
        face = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        # 모든 이미지를 48X48 사이즈로 리사이징
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
          # 감정 예측
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
        # frame에 나타날 label 위치 배정
        
        cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
        
        
 
        # label 수치화 후 출력
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            
            
            text = "{}: {:.2f}%".format(emotion, prob * 100)    
            w = int(prob * 300)
            cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
            
            
            
    # 윈도우 창 2개 열기 
    ## 감정 인식하는 카메라 창 - Emotion Recognition
    ## 감정의 확률을 나타내는 그래프 창 - Probabilities
    cv2.imshow('Emotion Recognition', frame)
    cv2.imshow("Probabilities", canvas)
    
    # 입력하면 P1 게임 결과 수집    
    if cv2.waitKey(33) & 0xFF == ord('1'):
        
        #화면 캡쳐 후 상위 폴더에 저장
        img=ImageGrab.grab(bbox=(0,0,900,900))
        saveas='p1.png'
        img.save(saveas)
        
        #정해둔 감정에 맞는 수치 출력
        if emo=='AN':
                x=preds[0]*100
                
        elif emo=='HA':
                x=preds[3]*100
                
        elif emo=='SA':
                x=preds[4]*100
        elif emo=='SU':
                x=preds[5]*100
                
        print("P1 DONE")
    

    elif cv2.waitKey(1) & 0xFF == ord('2'):
        
        img=ImageGrab.grab(bbox=(0,0,900,900))
        saveas='p2.png'
        img.save(saveas)
        
        if emo=='AN':
                y=preds[0]*100
               
        elif emo=='HA':
                y=preds[3]*100
                
        elif emo=='SA':
                y=preds[4]*100
               
        elif emo=='SU':
                y=preds[5]*100
        
        print("P2 DONE")
                
        break

#프로그램을 중단하고 모든 창 닫기
cv2.destroyAllWindows()

import turtle as t




#p1 게임 결과 거북이
t.shape("turtle")
t.pensize(10)
t.color('red')
t.penup()
t.goto(-300,100)
t.pendown()
t.fd(x*5)
if x>y:
    t.write("P1 %.2f WIN!" %x,False,"right",("",15))
else:
    t.write("P1 %.2f" %x,False,"right",("",15))

#p2 게임 결과 거북이

t.penup()
t.color("blue")
t.goto(-300,50)
t.pendown()
t.fd(y*5)
if y>x:
    t.write("P2 %.2f WIN!" %y,False,"right",("",15))
else:
    t.write("P2 %.2f" %y,False,"right",("",15))

#창 클릭하면 창 닫힘
turtle.exitonclick()

화남 = AN , 행복함 = HA, 슬픔 = SA, 놀람 = SU
대결할 감정:AN
P1 DONE
P2 DONE


Terminator: 

## 